<a href="https://colab.research.google.com/github/alexacheui/ateema_capstone/blob/main/typeform_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pymongo --quiet
!pip install flask pyngrok --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 669.1/669.1 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 5.4 MB/s eta 0:00:00


In [ ]:
from flask import Flask, request, jsonify
import pandas as pd
from pymongo import MongoClient
from datetime import datetime
import os
from google.colab import userdata
# import asyncio
# from pyngrok import ngrok

class SurveyApp:
    def __init__(self, mongo_uri):
        try:
            self.client = MongoClient(mongo_uri)
            self.db = self.client.ateema
            self.collection = self.db['survey']
            print("Successfully connected to MongoDB.")
        except Exception as e:
            print(f"Error connecting to MongoDB: {e}")
            raise

    def process_webhook(self, request_data):
        print("**Received Data:**")
        print(request_data)

        def join_labels(labels):
            return ', '.join([label.split()[0] for label in labels])

        try:
            answers = request_data['form_response']['answers']
            survey_data = {
                'first_name': answers[0]['text'],
                'last_name': answers[1]['text'],
                'phone_number': answers[2]['phone_number'],
                'email': answers[3]['email'],
                'travel_city': answers[4]['choice']['label'],
                'start_date': answers[5]['date'],
                'end_date': answers[6]['date'],
                'group_type': answers[7]['choice']['label'],
                'group_size': answers[8]['number'],
                'group_age': answers[9]['choice']['label'],
                'pref_dining': join_labels(answers[10]['choices']['labels']),
                'pref_beverage': join_labels(answers[11]['choices']['labels']),
                'pref_entertainment': join_labels(answers[12]['choices']['labels']),
                'pref_cultural': join_labels(answers[13]['choices']['labels']),
                'pref_outdoor': join_labels(answers[14]['choices']['labels']),
                'pref_education': join_labels(answers[15]['choices']['labels']),
                'pref_shop': join_labels(answers[16]['choices']['labels']),
                'comment': answers[17]['text'],
                'video_length': answers[18]['choice']['label'],
                'timestamp': datetime.utcnow()
            }

            df = pd.DataFrame([survey_data])
            print("**Created DataFrame:**")
            print(df)

            records = df.to_dict(orient='records')
            self.collection.insert_many(records)

        except Exception as e:
            print(f"Error processing data: {e}")

    def get_latest_survey_data(self):
        try:
            latest_data = self.collection.find().sort('timestamp', -1).limit(1)
            latest_data = list(latest_data)
            if latest_data:
                # Convert ObjectId to string
                for data in latest_data:
                    data['_id'] = str(data['_id'])

                df = pd.DataFrame(latest_data)
                return df.to_json(orient='records')
            else:
                return 'No survey data available yet.'
        except Exception as e:
            print(f"Error retrieving latest survey data: {e}")
            return 'Internal server error.', 500

app = Flask(__name__)
# app.config['DEBUG'] = True  # Enable debug mode
mongo_uri = userdata.get('mongodb_key')
survey_app = SurveyApp(mongo_uri)


@app.route("/")
def index():
    return "<h1>Hello World</h1>"

@app.route('/webhook', methods=['POST'])
def typeform_webhook():
    request_data = request.get_json()
    survey_app.process_webhook(request_data)
    return 'Data received successfully.'

@app.route('/get_latest_data', methods=['GET'])
def get_latest_survey_data():
    result = survey_app.get_latest_survey_data()
    return result if isinstance(result, str) else jsonify(result)

if __name__ == '__main__':
    ngrok.set_auth_token(userdata.get('ngrok_key'))
    ngrok_tunnel = ngrok.connect(5000)
    print('Public URL:', ngrok_tunnel.public_url)
    app.run(port=5000)


Successfully connected to MongoDB.
Public URL: https://f1f1-35-229-241-137.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


**Received Data:**
{'event_id': '01J16M9WFJ41297Y98F3XT5HTD', 'event_type': 'form_response', 'form_response': {'form_id': 'FkcBezxr', 'token': '01J16M9WFJ41297Y98F3XT5HTD', 'landed_at': '2024-06-25T02:41:22Z', 'submitted_at': '2024-06-25T02:41:22Z', 'definition': {'id': 'FkcBezxr', 'title': 'My new form', 'fields': [{'id': 'cyC2jup2h9qd', 'ref': '596e32db-f5ef-422a-85b0-1e64723cff81', 'type': 'short_text', 'title': 'First name', 'properties': {}}, {'id': 'Ap9DQevjbydK', 'ref': '8b4f0707-c312-4658-94b0-d215a63fced2', 'type': 'short_text', 'title': 'Last name', 'properties': {}}, {'id': 'eZm0OtqIP460', 'ref': '600b2743-94a4-49bd-a9e9-2651313a9c5f', 'type': 'phone_number', 'title': 'Phone number', 'properties': {}}, {'id': 'RWH1jVaQrbdz', 'ref': 'dfe478d9-e2e2-4e75-9623-6280b9567724', 'type': 'email', 'title': 'Email', 'properties': {}}, {'id': 'rAKa2WQMixJL', 'ref': '86a2d1bb-5371-47bd-9e59-8556125ea162', 'type': 'picture_choice', 'title': '*Travel is always exciting! which town are you 

INFO:werkzeug:127.0.0.1 - - [25/Jun/2024 02:41:23] "POST /webhook HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Jun/2024 02:41:58] "GET /apple-touch-icon-precomposed.png HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [25/Jun/2024 02:41:58] "GET /apple-touch-icon.png HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [25/Jun/2024 02:41:58] "GET /favicon.ico HTTP/1.1" 404 -
